In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

In [2]:
# Carregar os dados
solicitacoes_credito = pd.read_csv('Arquivos/solicitacoescredito.csv')


In [3]:
solicitacoes_credito2 = solicitacoes_credito.drop(columns=['scorePontualidade'])

In [4]:
solicitacoes_credito3 = solicitacoes_credito2


In [37]:
max_atraso = solicitacoes_credito3['maiorAtraso'].max()

# Calcular o score normalizado
solicitacoes_credito3['score_atraso'] = 1 - (solicitacoes_credito3['maiorAtraso'] / max_atraso)

# Garantir que o score esteja no intervalo [0, 1]
solicitacoes_credito3['score_atraso'] = np.clip(solicitacoes_credito3['score_atraso'], 0, 1)

solicitacoes_credito3.head()

# print(solicitacoes_credito3.columns)

,numero_solicitacao,razaoSocial,nomeFantasia,cnpjSemTraco,maiorAtraso,margemBrutaAcumulada,percentualProtestos,primeiraCompra,prazoMedioRecebimentoVendas,titulosEmAberto,...,periodoDemonstrativoEmMeses,custos,anoFundacao,intervaloFundacao,capitalSocial,restricoes,empresa_MeEppMei,limiteEmpresaAnaliseCredito,dataAprovadoNivelAnalista,score_atraso
0,1,James Richardson-Patel,Alexandra Williams,KEBE17609492220843,0,0.252448,0.0,2015-12-10T00:00:00,0,0.00,...,12.0,0.0,2003.0,Acima de 17 anos,90000.0,False,True,43200.0,2020-02-03T20:57:33,1.000000
1,2,Dr. Geoffrey Walsh,Mr. Darren Arnold,JRBK88908250677300,0,0.000000,0.0,2019-06-12T17:28:31,0,0.00,...,12.0,0.0,2003.0,Acima de 17 anos,90000.0,False,True,43200.0,2020-02-03T20:57:33,1.000000
2,3,Joanna Hudson,Dr. David Rees,GCVQ28531614261293,4,0.624777,0.0,2019-11-27T00:00:00,0,0.00,...,7.0,0.0,2014.0,De 6 a 10 anos,20000.0,False,True,4320.0,2020-02-04T16:40:49,0.996838
3,4,Gordon Jones-Hopkins,Sara Reid-Robson,KJND32266018316396,20,0.000000,0.0,2017-02-13T17:20:27,0,0.00,...,12.0,0.0,2013.0,De 6 a 10 anos,30000.0,False,True,5920.0,2020-02-04T16:37:52,0.984190
4,5,Nigel Lee,Dr. Stanley Duncan,CGQN15826802440348,20,0.454088,0.0,2010-07-13T00:00:00,20,1486.95,...,9.0,40680051.0,2002.0,Acima de 17 anos,75000.0,False,False,89000.0,2020-02-04T15:06:28,0.984190


In [52]:
# mostra a quantidade de nulos por coluna
nulos_por_coluna = solicitacoes_credito3.isnull().sum()


# Filtrar para manter apenas colunas com nulos diferentes de zero
colunas_com_nulos = nulos_por_coluna[nulos_por_coluna > 0]

print(colunas_com_nulos)


dataAprovadoEmComite    11
periodoBalanco           4
ativoCirculante          4
passivoCirculante        4
totalAtivo               4
endividamento            4
duplicatasAReceber       4
estoque                  4
dtype: int64


In [38]:
solicitacoes_credito4 = solicitacoes_credito3

moda_faturamento = solicitacoes_credito3['faturamentoBruto'].mode()[0]
moda_patrimonio = solicitacoes_credito3['totalPatrimonioLiquido'].mode()[0]

# Preencher valores nulos com a moda calculada
solicitacoes_credito4['faturamentoBruto'] = solicitacoes_credito4['faturamentoBruto'].fillna(moda_faturamento)
solicitacoes_credito4['totalPatrimonioLiquido'] = solicitacoes_credito4['totalPatrimonioLiquido'].fillna(moda_patrimonio)


# Preencher valores nulos com valores não nulos do mesmo CNPJ
solicitacoes_credito4 = solicitacoes_credito3.groupby('cnpjSemTraco').ffill().bfill()

solicitacoes_credito4.head()



,numero_solicitacao,razaoSocial,nomeFantasia,maiorAtraso,margemBrutaAcumulada,percentualProtestos,primeiraCompra,prazoMedioRecebimentoVendas,titulosEmAberto,valorSolicitado,...,periodoDemonstrativoEmMeses,custos,anoFundacao,intervaloFundacao,capitalSocial,restricoes,empresa_MeEppMei,limiteEmpresaAnaliseCredito,dataAprovadoNivelAnalista,score_atraso
0,1,James Richardson-Patel,Alexandra Williams,0,0.252448,0.0,2015-12-10T00:00:00,0,0.00,50000.0,...,12.0,0.0,2003.0,Acima de 17 anos,90000.0,False,True,43200.0,2020-02-03T20:57:33,1.000000
1,2,Dr. Geoffrey Walsh,Mr. Darren Arnold,0,0.000000,0.0,2019-06-12T17:28:31,0,0.00,10000.0,...,12.0,0.0,2003.0,Acima de 17 anos,90000.0,False,True,43200.0,2020-02-03T20:57:33,1.000000
2,3,Joanna Hudson,Dr. David Rees,4,0.624777,0.0,2019-11-27T00:00:00,0,0.00,20000.0,...,7.0,0.0,2014.0,De 6 a 10 anos,20000.0,False,True,4320.0,2020-02-04T16:40:49,0.996838
3,4,Gordon Jones-Hopkins,Sara Reid-Robson,20,0.000000,0.0,2017-02-13T17:20:27,0,0.00,25000.0,...,12.0,0.0,2013.0,De 6 a 10 anos,30000.0,False,True,5920.0,2020-02-04T16:37:52,0.984190
4,5,Nigel Lee,Dr. Stanley Duncan,20,0.454088,0.0,2010-07-13T00:00:00,20,1486.95,50000.0,...,9.0,40680051.0,2002.0,Acima de 17 anos,75000.0,False,False,89000.0,2020-02-04T15:06:28,0.984190


In [66]:

# Separar variáveis independentes e dependentes
colunas = ['score_atraso']
variaveis_independentes = [col for col in solicitacoes_credito3.columns if col not in colunas]
X = solicitacoes_credito3[variaveis_independentes]
y = solicitacoes_credito3['score_atraso']



In [67]:
X = pd.get_dummies(X)
X.head()

,numero_solicitacao,maiorAtraso,margemBrutaAcumulada,percentualProtestos,prazoMedioRecebimentoVendas,titulosEmAberto,valorSolicitado,diferencaPercentualRisco,percentualRisco,dashboardCorrelacao,...,dataAprovadoNivelAnalista_2021-02-25T20:04:53,dataAprovadoNivelAnalista_2021-02-25T20:05:32,dataAprovadoNivelAnalista_2021-02-25T20:06:19,dataAprovadoNivelAnalista_2021-02-25T20:06:47,dataAprovadoNivelAnalista_2021-02-25T20:07:50,dataAprovadoNivelAnalista_2021-02-25T20:10:15,dataAprovadoNivelAnalista_2021-02-25T20:13:33,dataAprovadoNivelAnalista_2021-02-25T20:28:20,dataAprovadoNivelAnalista_2021-02-25T20:33:36,dataAprovadoNivelAnalista_2021-02-25T20:40:58
0,1,0,0.252448,0.0,0,0.00,50000.0,0.716981,0.283019,0.000000,...,False,False,False,False,False,False,False,False,False,False
1,2,0,0.000000,0.0,0,0.00,10000.0,1.000000,0.000000,0.000000,...,False,False,False,False,False,False,False,False,False,False
2,3,4,0.624777,0.0,0,0.00,20000.0,0.716981,0.283019,0.000000,...,False,False,False,False,False,False,False,False,False,False
3,4,20,0.000000,0.0,0,0.00,25000.0,0.396226,0.603774,0.485811,...,False,False,False,False,False,False,False,False,False,False
4,5,20,0.454088,0.0,20,1486.95,50000.0,0.830189,0.169811,0.000000,...,False,False,False,False,False,False,False,False,False,False


In [55]:
X = X.bfill()


In [56]:
# Dividir os dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Normalizar os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [57]:
# Criar e treinar o modelo de Regressão Linear
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Fazer previsões
y_pred = model.predict(X_test_scaled)

# Avaliar o modelo
mse = mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error: {mse}')

Mean Squared Error: 0.0002462840417334044


In [58]:
# Obter os coeficientes
coeficientes = model.coef_
variaveis = X.columns

# Criar um DataFrame para visualizar coeficientes
coef_df = pd.DataFrame({
    'Variável': variaveis,
    'Coeficiente': coeficientes
})


In [59]:
# Ordenar as variáveis por coeficiente
coef_df = coef_df.sort_values(by='Coeficiente', ascending=False)

print("\nVariáveis mais e menos relevantes:")
print(coef_df)


Variáveis mais e menos relevantes:
                                Variável  Coeficiente
6062     cnpjSemTraco_FKWA23828790343443     0.000482
9866  primeiraCompra_2019-08-27T17:45:11     0.000482
3476   nomeFantasia_Dr. Kenneth McDonald     0.000442
2587            razaoSocial_Shaun Rowley     0.000435
2883              nomeFantasia_Amy Adams     0.000427
...                                  ...          ...
5960     cnpjSemTraco_EHDJ94994204173246    -0.002817
4264           nomeFantasia_Leslie Smith    -0.004689
7470     cnpjSemTraco_SJWK90115301924947    -0.004695
2693        razaoSocial_Terry King-Brown    -0.004707
1                            maiorAtraso    -0.009916

[17950 rows x 2 columns]


In [62]:
# Obter os coeficientes do modelo treinado
coeficientes = model.coef_

# Obter os nomes das variáveis (colunas) correspondentes
variaveis = X.columns

# Criar um DataFrame para associar variáveis e coeficientes
coef_df = pd.DataFrame({
    'Variável': variaveis,
    'Coeficiente': coeficientes
})

# Converter coeficientes para valores absolutos, caso queira a magnitude
# coef_df['Coeficiente_Absoluto'] = abs(coef_df['Coeficiente'])

# Ordenar as variáveis pelos coeficientes, do maior para o menor
coef_df = coef_df.sort_values(by='Coeficiente', ascending=True)

# Exibir o DataFrame com os coeficientes ordenados
print("\nVariáveis mais e menos relevantes:")
print(coef_df)



Variáveis mais e menos relevantes:
                                Variável  Coeficiente
1                            maiorAtraso    -0.009916
2693        razaoSocial_Terry King-Brown    -0.004707
7470     cnpjSemTraco_SJWK90115301924947    -0.004695
4264           nomeFantasia_Leslie Smith    -0.004689
5960     cnpjSemTraco_EHDJ94994204173246    -0.002817
...                                  ...          ...
2883              nomeFantasia_Amy Adams     0.000427
2587            razaoSocial_Shaun Rowley     0.000435
3476   nomeFantasia_Dr. Kenneth McDonald     0.000442
9866  primeiraCompra_2019-08-27T17:45:11     0.000482
6062     cnpjSemTraco_FKWA23828790343443     0.000482

[17950 rows x 2 columns]


In [65]:
# Saida excel
# Nome do arquivo Excel
arquivo_excel = 'coef.xlsx'

# Usar ExcelWriter com openpyxl
with pd.ExcelWriter(arquivo_excel, engine='openpyxl') as writer:
    coef_df.to_excel(writer, sheet_name='coeficiente', index=False)

In [18]:
# # Saida excel
# # Nome do arquivo Excel
# arquivo_excel = 'planilhas.xlsx'

# # Usar ExcelWriter com openpyxl
# with pd.ExcelWriter(arquivo_excel, engine='openpyxl') as writer:
#     cliente.to_excel(writer, sheet_name='cliente', index=False)
#     cliente_distinct.to_excel(writer, sheet_name='cliente_distinct', index=False)